## Example Usage of the GMail Python API

This Notebook shows the basic usage of the GMail API

1. Load E-Mails from the server

2. Send E-Mails with the server

In order to be able to use the API you need a valid token as a .pickle file. In the first initialization of the GMail service you need to pass the path to this token.pickle file. Change the parameter below to your file to rerun the examples. 


For furhter information on how to send attachments, ... please consult the Documentation: 

https://googleapis.github.io/google-api-python-client/docs/epy/index.html

and the Guide: 

https://developers.google.com/gmail/api/quickstart/python

In [ ]:
from googleapiclient.discovery import build, Resource
from email.mime.text import MIMEText
import base64
import pickle
import os
from typing import Dict
import email

### Basic Functions

In [ ]:
def load_service(token_path:str):
    """ Load the service from the specifies token file

    Args:
        token_path (str): Authorized Gmail API service instance.
    """
    if os.path.exists(token_path):
        with open(token_path, 'rb') as token:
            creds = pickle.load(token)
            service = build('gmail', 'v1', credentials=creds)
            return service

    else: 
        print("Credentials not found")
        return None

In [ ]:
def send_message(service:Resource, user_id:str, message:Dict[str,str]):
    """Send an email message.

    Args:
        service (Resource): Authorized Gmail API service instance.
        user_id (str): User's email address. The special value "me" can be used to indicate the authenticated user.
        message (Dict[str,str]): Message to be sent.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                  .execute())
        print('Message Id: %s' % message['id'])
    except Exception as e:
        print('An error occurred: %s' % e)

In [ ]:
def create_message(sender:str, to:str, subject:str, message_text:str) -> Dict[str,str]:
  """Create a message for an email.

  Args:
    sender (str): Email address of the sender.
    to (str): Email address of the receiver.
    subject (str): The subject of the email message.
    message_text (str): The text of the email message.

  Returns:
    Dict[str,str]: An object containing a base64url encoded email object.
  """
  message = MIMEText(message_text, "html")
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  encoded_message = base64.urlsafe_b64encode(message.as_bytes())
  return {'raw': encoded_message.decode()}


In [ ]:
def recieve_unread_messages(service:Resource):
    """ Recieve all unread messages and print them

        Args:
        service (Resource): Authorized Gmail API service instance.

    """
    messages = []
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # iterate through all the messages 
    for msg in result.get('messages') : 
        # Get the message from its id   
        # Use try-except to avoid any Errors 
        message = service.users().messages().get(userId="me", id=msg["id"],
                                             format='raw').execute()
                                             
        if "UNREAD" in message["labelIds"]:
            try: 
                msg_str = base64.urlsafe_b64decode(message['raw'].encode("utf-8")).decode("utf-8")
                mime_msg = email.message_from_string(msg_str)

                messages.append(mime_msg)
    
            except: 
                print("error")
                pass
    return messages

In [ ]:
def print_message(message):
    """ Print the Content of a Message

    Args:
        message (email): The email encoded message
    """

    if message.is_multipart():
        for payload in message.get_payload():
            print(payload.get_payload())
    else:
        print(message.get_payload())

### Usage of the function

In [ ]:
token_path = "token.pickle"

In [ ]:
service = load_service(token_path)

In [ ]:
message = create_message("datathon@lumos-consulting.at", "datathon@lumos-consulting.at", "Test_0", "Das ist der erste Test")

In [ ]:
send_message(service, "me", message)

Message Id: 17702f1bcdd7e9b3


In [ ]:
messages = recieve_unread_messages(service)

In [ ]:
for message in messages: 
    print("New Message!")
    print_message(message)
    print()


New Message!
Das ist der erste Test

New Message!
Das ist der erste Test

